In [322]:
import pandas as pd
import numpy as np
import jsonlines
import json
import os
import re
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [323]:
optionlim = 0.1
predfolder = 'multitask4mixbidspec2verbose2_prin_ml_1024_50_bs50_seq3_4bs_2DAYS' ## folder containing pred file

goldfolder = 'multitask4mix_bi' ## folder containing gold file

dataset = 'ml' ## dataset code (or part of it), is a column in gold file

gfile = goldfolder+'/testml.csv' ## gold file
#gfile = goldfolder+'/test.csv'

#opfile = goldfolder+'/testmavenevex.csv'

task = 'EV-EX'



add_to_main_file = False


golddf1 = pd.read_csv(gfile)

if('subtask' in golddf1.columns):
    golddf = golddf1[golddf1.subtask==task]
    golddf = golddf[golddf.dataset.str.contains(dataset)]
    print(golddf.shape)
else:
    golddf = golddf1

return_seqs = 3

textcol = 'text'
relcol = 'src_label'

#if('src21' in predfolder):
#    relcol = 'src_label21'
    


(875, 15)


In [324]:


def reverse_21(pred):
    ## reversed outputs/ convert to src_label1 format
    predz=[]
    for p in pred:
        fin=[]
        events = [x.strip() for x in p.split('|')]
        for e in events:
            #print(e)
            if('->' not in e):
                typ,trigs=['','']
            else:
                typ,trigs=e.split('->')[0:2]
            
            for tg in trigs.split('@'):
                fin.append(tg+'->'+typ)
        if(p.strip()=='->'):
            predz.append('NONE')
        else:
            predz.append(' | '.join(fin))
        
    pred = predz.copy()
    return pred

    

    


def reverse_12(pred):
    ## reversed outputs/ convert to src_label1 format
    predz=[]
    for idx,p in enumerate(pred):
        nm=[]
        if('NONE' in p):
            predz.append('NONE')
            continue
        
        events = [x.strip() for x in p.split('|')]
        #print(events,sb.iloc[idx])
        for e in events:
            if('->' not in e):
                continue
            tg = e.split('->')[0]
            tps = e.split('->')[1].split('@')
            nm += [tg+'->'+p for p in tps]
        if(len(nm)==0):
            nm = ['NONE']
        predz.append(' | '.join(nm))
        
    pred = predz.copy()
    return pred

    


In [325]:
gold = golddf[relcol]
print(len(gold))

875


In [326]:
pred = open('results/'+predfolder+'/generated_predictions.txt').readlines()
print(len(pred))
pred.append('\n')
print(len(pred))

10499
10500


In [327]:
pred[0:5],gold[0:5]

(["b'The types of events in this text are negative_regulation, localization, gene_expression, growth, positive_regulation, planned_process'\n",
  "b'The types of events in this text are localization, negative_regulation, gene_expression, growth, positive_regulation, planned_process'\n",
  "b'The types of events in this text are negative_regulation, growth, gene_expression, localization, positive_regulation, planned_process'\n",
  '\n',
  "b'The event triggers in this text are mediated, inhibition, growth, metastases, transfer, expression'\n"],
 2     inhibition->negative_regulation | transfer->pl...
 5     Inhibition->negative_regulation | angiogenesis...
 8                  angiogenic->blood_vessel_development
 11    angiogenic->blood_vessel_development | express...
 14    suppression->negative_regulation | induced->po...
 Name: src_label, dtype: object)

In [328]:
if('verb' in predfolder):
    p1 = pd.Series(pred).str.replace('The types of events in this text are ','').str.replace('The event triggers in this text are ','').str.replace('Events in this text: ','').str.replace('The event triggers and their types in this text are: ','')
    p2 = p1.str.replace(' is a trigger of type ','->').str.replace(',',' |')
    pred = p2

In [329]:
pls=[]

plsi=[]
for p in pred:
    
    if(p=='\n'):
        pls.append(plsi)
        plsi=[]
    else:
        plsi.append(p[2:-2])
    



In [330]:
#pls[0]


In [331]:
len(pls),len(pls[0]),len(gold)

(2625, 3, 875)

In [332]:
pls[0:4]

[['negative_regulation | localization | gene_expression | growth | positive_regulation | planned_process',
  'localization | negative_regulation | gene_expression | growth | positive_regulation | planned_process',
  'negative_regulation | growth | gene_expression | localization | positive_regulation | planned_process'],
 ['mediated | inhibition | growth | metastases | transfer | expression',
  'mediated | loss | growth | inhibition | transfer | expression',
  'mediated | inhibition | growth | transfer | expression'],
 ['growth->growth | mediated->regulation | inhibition->negative_regulation | expression->gene_expression | metastases->localization | transfer->planned_process | results->positive_regulation',
  'growth->growth | inhibition->negative_regulation | expression->gene_expression | mediated->positive_regulation | metastases->localization | transfer->planned_process | results->positive_regulation',
  'growth->growth | inhibition->negative_regulation | expression->gene_expression 

In [333]:
len(golddf),len(golddf1)

(875, 2625)

In [334]:
golddf.index, len(pls)

(Int64Index([   2,    5,    8,   11,   14,   17,   20,   23,   26,   29,
             ...
             2597, 2600, 2603, 2606, 2609, 2612, 2615, 2618, 2621, 2624],
            dtype='int64', length=875),
 2625)

In [335]:
if(len(golddf)<len(golddf1)):
    pls = np.asarray(pls)
    pls = pls[golddf.index]

In [336]:
pls = np.asarray(pls)
pls.shape

(875, 3)

In [337]:
op = pd.DataFrame(data={'text':golddf[textcol],'gold':gold})

In [338]:
for k in range(0,return_seqs):
    op['top_'+str(k+1)] = pls[:,k]
    if('src12' in predfolder):
        op['top_'+str(k+1)] = reverse_12(op['top_'+str(k+1)])
    if('src21' in predfolder):
        op['top_'+str(k+1)] = reverse_21(op['top_'+str(k+1)])
        
        
    
    if('verb' in predfolder):
        p1 = op['top_'+str(k+1)].str.replace('The types of events in this text are ','').str.replace('The event triggers in this text are ','').str.replace('Events in this text: ','').str.replace('The event triggers and their types in this text are: ','')
        p2 = p1.str.replace(' is a trigger of type ','->').str.replace(',',' |')
        op['top_'+str(k+1)] = p2

In [339]:
op.head()

,text,gold,top_1,top_2,top_3
2,EV-EX</s>mlee_processed_sent</s>Adenovirus-med...,inhibition->negative_regulation | transfer->pl...,growth->growth | mediated->regulation | inhibi...,growth->growth | inhibition->negative_regulati...,growth->growth | inhibition->negative_regulati...
5,EV-EX</s>mlee_processed_sent</s>Inhibition of ...,Inhibition->negative_regulation | angiogenesis...,Inhibition->negative_regulation | therapy->pla...,therapy->planned_process | Inhibition->negativ...,Inhibition->negative_regulation | angiogenesis...
8,"EV-EX</s>mlee_processed_sent</s>However, its w...",angiogenic->blood_vessel_development,angiogenic->blood_vessel_development,production->gene_expression | angiogenic->bloo...,produce->gene_expression | angiogenic->blood_v...
11,EV-EX</s>mlee_processed_sent</s>This limitatio...,angiogenic->blood_vessel_development | express...,angiogenic->blood_vessel_development | express...,expression->gene_expression | angiogenic->bloo...,express->gene_expression | angiogenic->blood_v...
14,EV-EX</s>mlee_processed_sent</s>We have constr...,suppression->negative_regulation | induced->po...,expresses->gene_expression | induced->positive...,induced->positive_regulation | expresses->gene...,expresses->gene_expression | induce->positive_...


In [340]:
def div(a,b):
    if b==0:
        return 0;
    else:
        return a/b

def scoring(gold,pred,task,indices=None,verbose=True,skipneg=True):
    tp = 0
    predn=0
    goldn=0
    skipped=0
    
    x=0
    cor=[]
    posind=[]

    for ind,row in enumerate(pred):
        #row = row.replace(': ','->')
        
        if(row=='->' or 'NONE' in row):
            row='NONE'
        if(indices is not None):
            gold2 = gold[indices[x]]
        x+=1
        
        if(str(gold2)=='nan'):
            gold2=''
        
        '''
        if(gold2=='NONE'):
            continue
        '''
        
        posind.append(ind)    
        row = row.split('|')
        gold2 = gold2.split('|')
        
        
        gold2 = [g.strip() for g in gold2]
        row = [g.strip() for g in row]
        
        r2 = row.copy()
        g2 = gold2.copy()
        
        #print(gold2,'\n\n',row)
        #break
        r2 = [r for r in row if len(r)>1]
        
        
        #gold[i] = str(gold[i]).split(' |')
        
        if(task=='EV-CLF'):
            row = list(set(row))
            gold2 = list(set(gold2))
            
        row.sort()
        gold2.sort()
        cor.append(row==gold2)
        
        if(skipneg and gold2[0]=='NONE'):
            skipped+=1
            continue

        
        for elem in row:
            if(len(elem)<1):
                continue
            
                
            
            if(elem in gold2): # exact phrase present
                tp+=1

            
                
                # NEW
                gold2.remove(elem)
            
        gold2 = g2.copy()
        for g in gold2: # part of pred phrase present in gold
            if(len(g)<1):
                continue
            
        
        predn+=len(row)-len([p for p in row if len(p)<1])
        goldn+=len(gold2)-len([p for p in gold2 if len(p)<1])
        #print('tp: ',tp,' predn: ',predn,' goldn: ',goldn)
        #print('tp_rel: ',tp_relaxed)
        #print('tp_rel2: ',tp_relaxed2)

    p = div(tp,predn)
    r = div(tp,goldn)
    f1 = div(2*p*r,(p+r))

    if(verbose):
        print('skipped: ',skipped)
        print('\nPrecision: ',p,'\nRecall: ',r,'\nF1-score: ',f1)

    
    '''
    prel = div(tp_relaxed,predn)
    rrel = div(tp_relaxed,goldn)
    f1rel = div(2*prel*rrel,(prel+rrel))
    
    print('\nRELAXED SCORING 1\nPrecision: ',prel,'\nRecall: ',rrel,'\nF1-score: ',f1rel)

    prel2 = div(tp_relaxed2,predn)
    rrel2 = div(tp_relaxed2,goldn)
    f1rel2 = div(2*prel2*rrel2,(prel2+rrel2))

    print('\nRELAXED SCORING 2\nPrecision: ',prel2,'\nRecall: ',rrel2,'\nF1-score: ',f1rel2)
    '''
    return posind,cor,p,r,f1

In [341]:
corlist=[]
for k in range(0,return_seqs):
    print('\n**************\nK is ',str(k+1))
    posind,cor,_,_,_ = scoring(op['gold'],op['top_'+str(k+1)],indices=op.index,task=task,skipneg=True)
    print('\nWITH NEGATIVE')
    posind,cor,_,_,_ = scoring(op['gold'],op['top_'+str(k+1)],indices=op.index,task=task,skipneg=False)
    corlist.append(cor)


**************
K is  1
skipped:  172

Precision:  0.7433284813197477 
Recall:  0.7580405739732805 
F1-score:  0.7506124448799608

WITH NEGATIVE
skipped:  0

Precision:  0.7357651245551602 
Recall:  0.7542179662562699 
F1-score:  0.7448772798919163

**************
K is  2
skipped:  172

Precision:  0.6773465703971119 
Recall:  0.7427016328550222 
F1-score:  0.7085201793721971

WITH NEGATIVE
skipped:  0

Precision:  0.6623913942904427 
Recall:  0.7300501595987232 
F1-score:  0.6945770065075921

**************
K is  3
skipped:  172

Precision:  0.6360895779500431 
Recall:  0.7308263236021771 
F1-score:  0.6801749942436104

WITH NEGATIVE
skipped:  0

Precision:  0.622626582278481 
Recall:  0.7177382580939352 
F1-score:  0.6668078796865071


In [342]:
## combining top1 and top2
a2 = [p+q for p,q in zip(op['top_1'].str.split('|'),op['top_2'].str.split('|'))]# + [p for p in op['top_2'].str.split('|')]
az=[]
for a in a2:
    az.append([p.strip() for p in a])
a2 = az
a2 = [list(set(a)) for a in a2]
a2 = pd.Series(a2).str.join(' | ')
op['top_2_combined'] = a2
len(a2)

875

In [343]:
## combining top1 and top2
posind,cor,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)
print('WITH NEGATIVE')
posind,cor,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)

skipped:  172

Precision:  0.6701461377870563 
Recall:  0.7941613062840178 
F1-score:  0.7269021739130435
WITH NEGATIVE
skipped:  0

Precision:  0.6622086358425678 
Recall:  0.7902416780665754 
F1-score:  0.7205821205821205


In [344]:
'''## combining top1 and top2 and top3
a3 = [p+q for p,q in zip(op['top_2_combined'].str.split('|'),op['top_3'].str.split('|'))]# + [p for p in op['top_2'].str.split('|')]
az=[]
for a in a3:
    az.append([p.strip() for p in a])
a3 = az
a3 = [list(set(a)) for a in a3]
a3 = pd.Series(a3).str.join(' | ')
op['top_3_combined'] = a3
len(a3)'''

"## combining top1 and top2 and top3\na3 = [p+q for p,q in zip(op['top_2_combined'].str.split('|'),op['top_3'].str.split('|'))]# + [p for p in op['top_2'].str.split('|')]\naz=[]\nfor a in a3:\n    az.append([p.strip() for p in a])\na3 = az\na3 = [list(set(a)) for a in a3]\na3 = pd.Series(a3).str.join(' | ')\nop['top_3_combined'] = a3\nlen(a3)"

In [345]:
'''## combining top1 and top2 and top3
posind,cor,_,_,_ = scoring(op['gold'],a3,indices=op.index,task=task,skipneg=True)
print('WITH NEGATIVE')
posind,cor,_,_,_ = scoring(op['gold'],a3,indices=op.index,task=task,skipneg=False)'''

"## combining top1 and top2 and top3\nposind,cor,_,_,_ = scoring(op['gold'],a3,indices=op.index,task=task,skipneg=True)\nprint('WITH NEGATIVE')\nposind,cor,_,_,_ = scoring(op['gold'],a3,indices=op.index,task=task,skipneg=False)"

In [346]:
op['cor1'] = corlist[0]
op['cor2'] = corlist[1]
op['cor3'] = corlist[2]

In [347]:
optionlim=0.1

## option 1. top 1 + lower(.5*top2)
a2=[]
for i,r in op.iterrows():
    p = r['top_1'].split('|')
    q = r['top_2'].split('|')
    len2 = int(np.floor(len(q)*optionlim))
    a = p+q[0:len2]   
    
    az=[]
    for k in a:
        az.append(k.strip())
    
    az = list(set(az))
    a2.append(' | '.join(az))

posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)
posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)
op['option_1'] = a2
op['cor_option_1'] = corr

skipped:  172

Precision:  0.7539370078740157 
Recall:  0.7580405739732805 
F1-score:  0.7559832223044658
skipped:  0

Precision:  0.7453808021631365 
Recall:  0.7542179662562699 
F1-score:  0.7497733454215775


In [348]:
optionlim = 0.4

## option 2. top 2 + upper(.5*top2)
a2=[]
for i,r in op.iterrows():
    p = r['top_1'].split('|')
    q = r['top_2'].split('|')
    len2 = int(np.ceil(len(q)*optionlim))
    a = p+q[0:len2]   
    
    az=[]
    for k in a:
        az.append(k.strip())
    
    az = list(set(az))
    a2.append(' | '.join(az))

posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)    
posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)
op['option_2'] = a2
op['cor_option_2'] = corr

skipped:  172

Precision:  0.7012929112795363 
Recall:  0.7783275606135577 
F1-score:  0.7378048780487805
skipped:  0

Precision:  0.6933115823817292 
Recall:  0.7751937984496124 
F1-score:  0.7319698600645856


In [349]:
## option 3. if top 1 in top 2, take top 2. else take top 1
a2=[]
for i,r in op.iterrows():
    if(r['top_1'] in r['top_2']):
        a2.append(r['top_2'])
    else:
        a2.append(r['top_1'])
        
posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)
posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)
op['option_3'] = a2
op['cor_option_3'] = corr

skipped:  172

Precision:  0.7015873015873015 
Recall:  0.7654626422563088 
F1-score:  0.7321344060577378
skipped:  0

Precision:  0.6949646275488972 
Recall:  0.7615139078887369 
F1-score:  0.7267188859878155


In [350]:
'''optionlim=0.2
    
## option 4. top 1 + lower(.5*top2)
a2=[]
for i,r in op.iterrows():
    p = r['top_1'].split('|')
    q = r['top_2'].split('|')
    len2 = int(np.floor(len(q)*optionlim))
    a = p+q[-len2:]   
    
    az=[]
    for k in a:
        az.append(k.strip())
    
    az = list(set(az))
    a2.append(' | '.join(az))

posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)
posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)
op['option_4'] = a2
op['cor_option_4'] = corr'''

"optionlim=0.2\n    \n## option 4. top 1 + lower(.5*top2)\na2=[]\nfor i,r in op.iterrows():\n    p = r['top_1'].split('|')\n    q = r['top_2'].split('|')\n    len2 = int(np.floor(len(q)*optionlim))\n    a = p+q[-len2:]   \n    \n    az=[]\n    for k in a:\n        az.append(k.strip())\n    \n    az = list(set(az))\n    a2.append(' | '.join(az))\n\nposind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)\nposind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)\nop['option_4'] = a2\nop['cor_option_4'] = corr"

In [351]:
'''optionlim=0.4

## option 5. top 2 + upper(.5*top2)
a2=[]
for i,r in op.iterrows():
    p = r['top_1'].split('|')
    q = r['top_2'].split('|')
    len2 = int(np.ceil(len(q)*optionlim))
    a = p+q[-len2:]   
    
    az=[]
    for k in a:
        az.append(k.strip())
    
    az = list(set(az))
    a2.append(' | '.join(az))

posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)    
posind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)
op['option_5'] = a2
op['cor_option_5'] = corr'''

"optionlim=0.4\n\n## option 5. top 2 + upper(.5*top2)\na2=[]\nfor i,r in op.iterrows():\n    p = r['top_1'].split('|')\n    q = r['top_2'].split('|')\n    len2 = int(np.ceil(len(q)*optionlim))\n    a = p+q[-len2:]   \n    \n    az=[]\n    for k in a:\n        az.append(k.strip())\n    \n    az = list(set(az))\n    a2.append(' | '.join(az))\n\nposind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=True)    \nposind,corr,_,_,_ = scoring(op['gold'],a2,indices=op.index,task=task,skipneg=False)\nop['option_5'] = a2\nop['cor_option_5'] = corr"

In [352]:
op.to_csv('results/'+predfolder+'/compare_top_'+str(return_seqs)+'.csv',index=False)

In [353]:
predfolder

'multitask4mixbidspec2verbose2_prin_ml_1024_50_bs50_seq3_4bs_2DAYS'

In [354]:
'''if(add_to_main_file):
    
    odf = pd.read_csv(opfile)
    
    if('evclf' in predfolder):
        odf['text'] = odf['text'] + ['</s>EVENT_TYPES</s>']*len(odf) + op['top_2_combined']
        odf['text_in'] = odf['text_in'] + ['</s>EVENT_TYPES</s>']*len(odf) + op['top_2_combined']
    elif('evid' in predfolder):
        odf['text'] = odf['text'] + ['</s>EVENT_TRIGGERS</s>']*len(odf) + op['top_2_combined']
        odf['text_in'] = odf['text_in'] + ['</s>EVENT_TRIGGERS</s>']*len(odf) + op['top_2_combined']
    elif('evex' in predfolder):
        odf['text'] = odf['text'] + ['</s>POSSIBLE EVENTS</s>']*len(odf) + op['top_2_combined']
        odf['text_in'] = odf['text_in'] + ['</s>POSSIBLE EVENTS</s>']*len(odf) + op['top_2_combined']
        
    
    odf.to_csv(opfile2,index=False)
    num = int(0.9*len(odf))
    
    af = opfile2.split('.')[0]+'_90p.csv'
    odf[0:num].to_csv(af,index=False)
    bf = opfile2.split('.')[0]+'_10p.csv'
    odf[num:].to_csv(bf,index=False)'''

"if(add_to_main_file):\n    \n    odf = pd.read_csv(opfile)\n    \n    if('evclf' in predfolder):\n        odf['text'] = odf['text'] + ['</s>EVENT_TYPES</s>']*len(odf) + op['top_2_combined']\n        odf['text_in'] = odf['text_in'] + ['</s>EVENT_TYPES</s>']*len(odf) + op['top_2_combined']\n    elif('evid' in predfolder):\n        odf['text'] = odf['text'] + ['</s>EVENT_TRIGGERS</s>']*len(odf) + op['top_2_combined']\n        odf['text_in'] = odf['text_in'] + ['</s>EVENT_TRIGGERS</s>']*len(odf) + op['top_2_combined']\n    elif('evex' in predfolder):\n        odf['text'] = odf['text'] + ['</s>POSSIBLE EVENTS</s>']*len(odf) + op['top_2_combined']\n        odf['text_in'] = odf['text_in'] + ['</s>POSSIBLE EVENTS</s>']*len(odf) + op['top_2_combined']\n        \n    \n    odf.to_csv(opfile2,index=False)\n    num = int(0.9*len(odf))\n    \n    af = opfile2.split('.')[0]+'_90p.csv'\n    odf[0:num].to_csv(af,index=False)\n    bf = opfile2.split('.')[0]+'_10p.csv'\n    odf[num:].to_csv(bf,index=Fa

In [355]:
odf.columns

NameError: name 'odf' is not defined